In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import KarateClub
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils.convert import from_networkx

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [56]:
class GAE_Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE_Encoder, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)
    
class VGAE_Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGAE_Encoder, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    
    #writer.add_scalar("loss", loss.item(), epoch)

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)


In [58]:
dataset = KarateClub()
data = dataset[0]

channels = 16
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())

# encoder: written by us; decoder: default (inner product)
model = pyg_nn.VGAE(VGAE_Encoder(dataset.num_features, channels)).to(dev)
labels = data.y
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)
x, train_pos_edge_index = data.x.to(dev), data.train_pos_edge_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train(epoch)
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    #writer.add_scalar("AUC", auc, epoch)
    #writer.add_scalar("AP", ap, epoch)
    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

CUDA availability: False
Epoch: 010, AUC: 0.5306, AP: 0.5525
Epoch: 020, AUC: 0.7143, AP: 0.6742


/Users/mateicosa/Bocconi/BIDSA/Network_Science/.conda/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch: 030, AUC: 0.8776, AP: 0.9123
Epoch: 040, AUC: 0.9592, AP: 0.9683
Epoch: 050, AUC: 0.8367, AP: 0.8274
Epoch: 060, AUC: 0.8367, AP: 0.8274
Epoch: 070, AUC: 0.8980, AP: 0.8736
Epoch: 080, AUC: 0.8980, AP: 0.8736
Epoch: 090, AUC: 0.8980, AP: 0.8736
Epoch: 100, AUC: 0.8980, AP: 0.8736
Epoch: 110, AUC: 0.8980, AP: 0.8736
Epoch: 120, AUC: 0.8980, AP: 0.8736
Epoch: 130, AUC: 0.8980, AP: 0.8736
Epoch: 140, AUC: 0.8980, AP: 0.8736
Epoch: 150, AUC: 0.9184, AP: 0.9094
Epoch: 160, AUC: 0.9184, AP: 0.9094
Epoch: 170, AUC: 0.9184, AP: 0.9094
Epoch: 180, AUC: 0.9184, AP: 0.9094
Epoch: 190, AUC: 0.9184, AP: 0.9094
Epoch: 200, AUC: 0.9184, AP: 0.9094


In [62]:
data = from_networkx(G)

channels = 16
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())

# encoder: written by us; decoder: default (inner product)
model = pyg_nn.GAE(GAE_Encoder(data.num_features, channels)).to(dev)
labels = data.y
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)
x, train_pos_edge_index = data.x.to(dev), data.train_pos_edge_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train(epoch)
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    #writer.add_scalar("AUC", auc, epoch)
    #writer.add_scalar("AP", ap, epoch)
    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

CUDA availability: False
Epoch: 010, AUC: 0.5000, AP: 0.5000
Epoch: 020, AUC: 0.5000, AP: 0.5000


/Users/mateicosa/Bocconi/BIDSA/Network_Science/.conda/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch: 030, AUC: 0.5000, AP: 0.5000
Epoch: 040, AUC: 0.5000, AP: 0.5000
Epoch: 050, AUC: 0.5000, AP: 0.5000
Epoch: 060, AUC: 0.5000, AP: 0.5000
Epoch: 070, AUC: 0.5000, AP: 0.5000
Epoch: 080, AUC: 0.5000, AP: 0.5000
Epoch: 090, AUC: 0.5000, AP: 0.5000
Epoch: 100, AUC: 0.5000, AP: 0.5000
Epoch: 110, AUC: 0.5000, AP: 0.5000
Epoch: 120, AUC: 0.5000, AP: 0.5000
Epoch: 130, AUC: 0.5000, AP: 0.5000
Epoch: 140, AUC: 0.5000, AP: 0.5000
Epoch: 150, AUC: 0.5000, AP: 0.5000
Epoch: 160, AUC: 0.5000, AP: 0.5000
Epoch: 170, AUC: 0.5000, AP: 0.5000
Epoch: 180, AUC: 0.5000, AP: 0.5000
Epoch: 190, AUC: 0.5000, AP: 0.5000
Epoch: 200, AUC: 0.5000, AP: 0.5000


In [63]:
data.x

tensor([[ 3.3939e+01,  6.7710e+01,  9.2422e+04,  ...,  1.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 4.1153e+01,  2.0168e+01,  5.0272e+04,  ...,  0.0000e+00,
          1.0000e+00,  0.0000e+00],
        [ 2.8034e+01,  1.6596e+00,  5.6311e+04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.5553e+01,  4.8516e+01,  9.2422e+04,  ...,  1.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.3134e+01,  2.7849e+01,  3.6337e+04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.9015e+01,  2.9155e+01,  3.6337e+04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])